In [1]:
import sympy
import pandas as pd
from tqdm import tqdm
from symxplorer.symbolic_solver.domains import ExperimentResult

# Loading the Results of VLSI CMMF Exploration

In [2]:
results = ExperimentResult("VLSI_CMMF_POST_EXPERIMENT")
dir_of_results = results.find_results_file("Runs/VLSI_CMMF_Automated_NA")
results = results.load(f"{dir_of_results[0]}/results.pkl")

In [3]:
results.classifications_dict.keys()

dict_keys(['Z1_Z2_Z3_Z4_Z5_Z6'])

In [4]:
classifications_of_intrest = results.classifications_dict["Z1_Z2_Z3_Z4_Z5_Z6"]
len(classifications_of_intrest), classifications_of_intrest[0]

(4096,
 FilterClassification(False, X-INVALID-ORDER, None)(R_1, R_2, R_3, R_4, R_5, R_6), R_1*R_2*R_4*R_6/(R_1*R_4*R_5 - R_2*R_3*R_4 + R_2*R_3*R_5 + R_2*R_4*R_5 + R_3*R_4*R_5))

In [5]:
# df = results.flatten_classifications() # This way we can access all the classsifcation into a pandas df

In [6]:
classifications_of_intrest = []

for key in results.classifications_dict.keys():
    classifications = results.classifications_dict[key]

    classifications_of_intrest += ([classification 
                                for classification in classifications 
                                if (classification.fType in ["BP"] )]
                                )
    
    fTypes = set(classification.fType for classification in classifications_of_intrest)

len(classifications_of_intrest), fTypes

(120, {'BP'})

In [7]:
idx = 3
classifications_of_intrest[idx].zCombo

(R_1, R_2, R_3, 1/(C_4*s), R_5 + 1/(C_5*s), R_6)

In [8]:
classifications_of_intrest[idx].transferFunc

C_5*R_1*R_2*R_6*s/(C_4*C_5*R_2*R_3*R_5*s**2 + R_1 + R_2 + R_3 + s*(C_4*R_2*R_3 + C_5*R_1*R_5 - C_5*R_2*R_3 + C_5*R_2*R_5 + C_5*R_3*R_5))

In [9]:
Q = classifications_of_intrest[idx].parameters["Q"].simplify()
Q

sqrt(C_4)*sqrt(C_5)*sqrt(R_2)*sqrt(R_3)*sqrt(R_5)*sqrt(R_1 + R_2 + R_3)/(C_4*R_2*R_3 + C_5*R_1*R_5 - C_5*R_2*R_3 + C_5*R_2*R_5 + C_5*R_3*R_5)

In [10]:
K = classifications_of_intrest[idx].parameters["K_BP"].simplify()
K

C_5*R_1*R_2*R_6/(C_4*R_2*R_3 + C_5*R_1*R_5 - C_5*R_2*R_3 + C_5*R_2*R_5 + C_5*R_3*R_5)

In [11]:
wo = classifications_of_intrest[idx].parameters["wo"].simplify()
wo

sqrt(R_1 + R_2 + R_3)/(sqrt(C_4)*sqrt(C_5)*sqrt(R_2)*sqrt(R_3)*sqrt(R_5))

# Examining the Higher Order Filters 

In [ ]:
classifications_of_intrest = results.get_filter_types(["X-INVALID-ORDER", "X-INVALID-NUMER"])
len(classifications_of_intrest)

## Update filterOrders

In [ ]:
count = {
    "X-CONST" : 0,
}
s = sympy.symbols("s")

for i, classification in enumerate(classifications_of_intrest):
    tf = classification.transferFunc

    if not (s in tf.free_symbols): # TF id NOT a function of S - Pure amplifier/level-shifter
        ftype = "X-CONST"
        count[ftype] += 1
        classifications_of_intrest[i].fType = "X-CONST"
        classifications_of_intrest[i].filterOrder = "Zero"

count

## Load the old classification into a classifier object

In [ ]:
from symxplorer.symbolic_solver.filter import Filter_Classifier

classifier = Filter_Classifier()
classifier.overwrite_classifications(classifications_of_intrest)
classifier.isClassified(), len(classifier.classifications)


In [21]:
# classifier.classifyFilter(filterOrder="FirstOrder")

In [22]:
# classifier.summarizeFilterType()

# Analyze higher orders
- **LP-3rd order**: num_deg = 0, denom_deg = 3
    -  The denom is decomposable into 1 second-orders and 1 first-order
- **HP-4th Order**: num_deg = 4, denom_deg = 4
    -  N(s) has only s^4 term
- **BP-4th order**: num_deg = 0, denom_deg = 4
    -  N(s) only has s^2 term
    -  The denom is decomposable into 2 second-orders

### LP-3rd Order

In [ ]:
from tqdm import tqdm
# from tqdm.gui import tqdm

def get_3rd_order_lp(classifier, check_for_stability: bool = False):
    classifications, count = classifier.findFilterInClassification(denom_order=3, numer_order=0, printMessage=False)
    print(f"{count} candidates for 3rd-order LP")

    output = []
    count = 0
    count_valid = 0
    for classification in tqdm(classifications, total=len(classifications)):
        count += 1
        tf = classification.transferFunc
        k, numer, denom = classifier.decompose_tf(tf)

        valid = True
        for poly in denom:
            order = poly.degree()
            if order == 1: 
                if (len(poly.as_dict()) != 2) or (check_for_stability and not classifier.is_poly_stable(poly)):
                    valid = False
                    break
                
            elif order == 2:
                if (len(poly.as_dict()) != 3) or (check_for_stability and not classifier.is_poly_stable(poly)):
                    valid = False
                    break

            else:
                valid = False
                break
        gain = None
        if valid:
            count_valid += 1
            tf = classification.transferFunc
            gain = sympy.limit(tf, sympy.symbols("s"), 0)
            # print(f"ID {count} - valid")

        output += [{
            "valid": valid,
            "zCombo": classification.zCombo,
            "tf" : sympy.latex(classification.transferFunc),
            "DC-Gain": gain,
            "k" :  k,
            "numer": numer,
            "denom": denom,
            "num-factor-count":len(numer),
            "denom-factor-count": len(denom)
        }]

    print(f"{count_valid} verified filters")

    return output
output_lp = get_3rd_order_lp(classifier)

df_lp = pd.DataFrame(output_lp)
df_lp.to_csv(f"Runs/{results.experiment_name}/3rd_LP.csv")
df_lp

In [ ]:
filtered = [output for output in output_lp if output["valid"]]
idx = 25
x = filtered[idx] 
x["DC-Gain"]

In [ ]:
x["denom"][1]

In [ ]:
x["denom"][0]

In [ ]:
x["zCombo"]

### HP-4th Order

In [ ]:

def get_4th_order_hp(classifier, check_for_stability: bool = False):
    classifications, count = classifier.findFilterInClassification(denom_order=4, numer_order=4, printMessage=False)
    print(f"{count} candidates for 4th-order HP")

    output = []
    count = 0
    count_valid = 0 
    for classification in tqdm(classifications, total=len(classifications)):
        count += 1
        tf = classification.transferFunc
        k, numer, denom = classifier.decompose_tf(tf)

        valid = True
        for poly in denom:
            order = poly.degree()
            if order == 1: 
                if (len(poly.as_dict()) != 2) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break
                
            elif order == 2:
                if (len(poly.as_dict()) != 3) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break

            else:
                valid = False
                break

        if len(numer) != 1:
            valid = False
        # else:
        #     for exp in numer[0].as_dict():
        #         if exp != 4:
        #             valid = False


        if valid:
            count_valid += 1
            # print(f"ID {count} - valid")

        output += [{
            "valid": valid,
            "zCombo": classification.zCombo,
            "classification" : classification,
            "k" :  k,
            "numer": numer,
            "denom": denom,
            "num-factor-count":len(numer),
            "denom-factor-count": len(denom)
        }]

    print(f"{count_valid} verified filters")

    return output

output_hp = get_4th_order_hp(classifier)

df_hp = pd.DataFrame(output_hp)
df_hp.to_csv(f"Runs/{results.experiment_name}/4th_HP.csv")
df_hp

### BP-4th Order

In [ ]:
classifications, count = classifier.findFilterInClassification(denom_order=4, numer_order=2, printMessage=False)

def get_4th_order_bp(classifier, check_for_stability: bool = False):
    classifications, count = classifier.findFilterInClassification(denom_order=4, numer_order=2, printMessage=False)
    print(f"{count} candidates for 4th-order BP")

    output = []
    count = 0
    count_valid = 0
    for classification in tqdm(classifications, total=len(classifications)):
        count += 1
        tf = classification.transferFunc
        k, numer, denom = classifier.decompose_tf(tf)

        valid = True
        for poly in denom:
            order = poly.degree()
            if order == 1: 
                if (len(poly.as_dict()) != 2) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break
                
            elif order == 2:
                if (len(poly.as_dict()) != 3) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break

            else:
                valid = False
                break

        if len(numer) != 1:
            valid = False

        if valid:
            count_valid += 1
            # print(f"ID {count} - valid")

        output += [{
            "valid": valid,
            "zCombo": classification.zCombo,
            "classification" : classification,
            "k" :  k,
            "numer": numer,
            "denom": denom,
            "num-factor-count":len(numer),
            "denom-factor-count": len(denom)
        }]

    print(f"{count_valid} verified filters")

    return output

output_bp = get_4th_order_bp(classifier)

df_bp = pd.DataFrame(output_bp)
df_bp.to_csv(f"Runs/{results.experiment_name}/4th_BP.csv")
df_bp

# Extract information from all

In [ ]:
data = []
for x in tqdm(classifier.classifications, total = len(classifier.classifications)):
    k, numer, denom = classifier.decompose_tf(x.transferFunc)
    
    degrees_list_denom = [poly.degree() for poly in denom]
    if len(degrees_list_denom) != 0:
        max_degrees_denom = max(degrees_list_denom)
    else: 
        max_degrees_denom = -1

    degrees_list_numer = [poly.degree() for poly in numer]

    if len(degrees_list_numer) != 0:
        max_degrees_numer = max(degrees_list_numer)
    else: 
        max_degrees_numer = -1

    data += [
        {
            "filter-ID" : x.filter_id,
            "combo" : x.zCombo,
            "TF" : sympy.latex(x.transferFunc),
            "param" : x.parameters,
            "type" : x.fType,

            "denom order" : x.tf_denom_order,
            "denom factor count" : len(denom),
            "denom factor degrees": degrees_list_denom,
            "max denom degree" : max_degrees_denom,

            "numer order" : x.tf_numer_order,
            "numer facator count" : len(numer),
            "numer factor degrees": degrees_list_numer,
            "max numer degree" : max_degrees_numer,

            "k" : k,
            "numer factors" : numer,
            "denom factors" :denom

        }
    ]

df = pd.DataFrame(data)
df.to_csv(f"Runs/{results.experiment_name}/consice_summary.csv")

# Playground

In [ ]:
from sympy import symbols, Poly, factor, simplify, solveset, S

def decompose_tf(transfer_function, variable, assumptions={}):
    """
    Decomposes the denominator of a transfer function into factors, 
    grouping strictly by the specified variable (e.g., `s`).

    Args:
        transfer_function: A Sympy expression representing the transfer function.
        variable: The main variable to group by (e.g., `s`).
        assumptions: A dictionary of assumptions for symbolic variables.

    Returns:
        A dictionary where each key is a factor (grouped by the main variable),
        and the value contains its roots and properties.
    """
    # Apply assumptions to all symbols
    for var, assumption in assumptions.items():
        var._assumptions.update({assumption: True})
    
    # Extract the denominator
    denominator = transfer_function.as_numer_denom()[1]
    
    # Ensure the denominator is treated as a polynomial in the given variable
    poly = Poly(denominator, variable)
    
    # Factorize strictly with respect to `variable`
    factored = factor(poly.as_expr())
    
    # Inspect each factor
    factors = factored.as_ordered_factors()
    results = {}
    for factor_expr in factors:
        # Solve for the roots of this factor
        roots = solveset(factor_expr, variable, domain=S.Complexes)
        
        # Analyze each root (e.g., real part)
        roots_info = []
        for root in roots:
            real_part = simplify(root.as_real_imag()[0])
            roots_info.append({
                "root": root,
                "real_part": real_part
            })
        
        # Store results for this factor
        results[str(factor_expr)] = {
            "factor": factor_expr,
            "roots": roots_info,
            "degree": Poly(factor_expr, variable).degree()
        }
    
    return results

# Example: Define symbolic components
R1, R2, R3 = symbols('R1 R2 R3', positive=True)  # Resistors
L1 = symbols('L1', positive=True)  # Inductor
C1, C2 = symbols('C1 C2', positive=True)  # Capacitors
s = symbols('s')

# Example transfer function
G = (s + R1) / (L1 * s**3 + (R2 + R3) * s**2 + 1 / C1 * s + 1 / C2)

# Decompose and inspect the factors
results = decompose_tf(G, s)
for factor, info in results.items():
    print(f"Factor: {info['factor']}")
    print(f"Degree: {info['degree']}")
    print("Roots:")
    for root_info in info["roots"]:
        print(f"  - Root: {root_info['root']}, Real part: {root_info['real_part']}")
    print()


In [ ]:
results.keys()

In [51]:
from sympy import symbols, Poly, factor, simplify, solveset, S

def decompose_tf(transfer_function):
    s = symbols("s")

    # Extract the numerator and denominator
    numerator, denominator = transfer_function.as_numer_denom()
 
    numerator = Poly(numerator, s)
    denominator = Poly(denominator, s)

    # Factorize strictly with respect to `variable`
    numerator = factor(numerator.as_expr())
    denominator = factor(denominator.as_expr())

    # Inspect each factor
    factors_numer = numerator.as_ordered_factors()
    factors_denom = denominator.as_ordered_factors()
    
    return factors_numer, factors_denom

# Example: Define symbolic components
R1, R2, R3 = symbols('R1 R2 R3', positive=True)  # Resistors
L1 = symbols('L1', positive=True)  # Inductor
C1, C2 = symbols('C1 C2', positive=True)  # Capacitors
s = symbols('s')

# Example transfer function
G = (s + R1) / (((s+R3)*(s**2 + s*2*C2+ 2*C2**2)).expand())
G.expand()

# Decompose and inspect the factors
tup = decompose_tf(G)
numer = tup[0]
denom = tup[1]


In [ ]:
from sympy import symbols, Eq, Matrix

# Define variables
x, y, z = symbols('x y z')

# Define nonlinear equations
eq1 = x
eq2 = z
eq3 = x/z

# Form the Jacobian matrix
equations = [eq1, eq2, eq3]
variables = [x, y, z]
jacobian = Matrix([[eq.diff(var) for var in variables] for eq in equations])

# Compute the rank of the Jacobian matrix
rank = jacobian.rank()

# Check if the rank equals the number of equations
print("Jacobian Matrix:")
print(jacobian)
print("Rank:", rank)
print("Number of equations:", len(equations))

if rank == len(equations):
    print("The system is independent.")
else:
    print("The system is dependent.")


# Sizing

## Loading

In [12]:
results.classifications_dict.keys()

dict_keys(['Z1_Z2_Z3_Z4_Z5_Z6'])

In [13]:
classifications_of_intrest = results.get_filter_types(["BP"])
len(classifications_of_intrest)

120

In [14]:
idx = 10
classification = classifications_of_intrest[idx]
classification.zCombo

(R_1, R_2, 1/(C_3*s), R_4/(C_4*R_4*s + 1), R_5, R_6/(C_6*R_6*s + 1))

In [15]:
classification.parameters

{'Q': sqrt(C_6)*sqrt(R_4)*sqrt(R_5)*sqrt(R_6)*sqrt(C_3*R_1 + C_3*R_2 + C_4*R_2)*sqrt(-R_2*R_4 + R_2*R_5 + R_4*R_5)/(C_3*R_1*R_4*R_5 + C_3*R_2*R_4*R_5 + C_4*R_2*R_4*R_5 - C_6*R_2*R_4*R_6 + C_6*R_2*R_5*R_6 + C_6*R_4*R_5*R_6),
 'wo': sqrt(-R_2*R_4 + R_2*R_5 + R_4*R_5)/sqrt(C_3*C_6*R_1*R_4*R_5*R_6 + C_3*C_6*R_2*R_4*R_5*R_6 + C_4*C_6*R_2*R_4*R_5*R_6),
 'bandwidth': (C_3*R_1*R_4*R_5 + C_3*R_2*R_4*R_5 + C_4*R_2*R_4*R_5 - C_6*R_2*R_4*R_6 + C_6*R_2*R_5*R_6 + C_6*R_4*R_5*R_6)/(sqrt(C_6)*sqrt(R_4)*sqrt(R_5)*sqrt(R_6)*sqrt(C_3*R_1 + C_3*R_2 + C_4*R_2)*sqrt(C_3*C_6*R_1*R_4*R_5*R_6 + C_3*C_6*R_2*R_4*R_5*R_6 + C_4*C_6*R_2*R_4*R_5*R_6)),
 'K_LP': 0,
 'K_HP': 0,
 'K_BP': C_3*R_1*R_2*R_4*R_6/(C_3*R_1*R_4*R_5 + C_3*R_2*R_4*R_5 + C_4*R_2*R_4*R_5 - C_6*R_2*R_4*R_6 + C_6*R_2*R_5*R_6 + C_6*R_4*R_5*R_6),
 'Qz': None,
 'Wz': None}

In [16]:
from symxplorer.designer_tools.sizing import Sizing_Assist

assistant = Sizing_Assist(filter_classification=classification)

2025-01-26 23:14:54,506 - matplotlib - matplotlib data path: /Users/danialnoorizadeh/.pyenv/versions/myenv/lib/python3.12/site-packages/matplotlib/mpl-data
2025-01-26 23:14:54,510 - matplotlib - CONFIGDIR=/Users/danialnoorizadeh/.matplotlib
2025-01-26 23:14:54,536 - matplotlib - interactive is False
2025-01-26 23:14:54,537 - matplotlib - platform is darwin
2025-01-26 23:14:54,570 - matplotlib - CACHEDIR=/Users/danialnoorizadeh/.matplotlib
2025-01-26 23:14:54,573 - matplotlib.font_manager - Using fontManager instance from /Users/danialnoorizadeh/.matplotlib/fontlist-v390.json


## Inspecting the problem space

In [17]:
assistant.get_design_variables()

dict_keys(['C_3', 'C_4', 'C_6', 'R_5', 'R_1', 'R_6', 'R_4', 'R_2'])

In [18]:
assistant.get_metrics()

dict_keys(['Q', 'wo', 'bandwidth', 'K_BP'])

In [19]:
assistant.metrics_dict

{'Q': sqrt(C_6)*sqrt(R_4)*sqrt(R_5)*sqrt(R_6)*sqrt(C_3*R_1 + C_3*R_2 + C_4*R_2)*sqrt(-R_2*R_4 + R_2*R_5 + R_4*R_5)/(C_3*R_1*R_4*R_5 + C_3*R_2*R_4*R_5 + C_4*R_2*R_4*R_5 - C_6*R_2*R_4*R_6 + C_6*R_2*R_5*R_6 + C_6*R_4*R_5*R_6),
 'wo': sqrt(-R_2*R_4 + R_2*R_5 + R_4*R_5)/(sqrt(C_6)*sqrt(R_4)*sqrt(R_5)*sqrt(R_6)*sqrt(C_3*R_1 + C_3*R_2 + C_4*R_2)),
 'bandwidth': (C_3*R_1*R_4*R_5 + C_3*R_2*R_4*R_5 + C_4*R_2*R_4*R_5 - C_6*R_2*R_4*R_6 + C_6*R_2*R_5*R_6 + C_6*R_4*R_5*R_6)/(C_6*R_4*R_5*R_6*sqrt(C_3**2*R_1**2 + 2*C_3**2*R_1*R_2 + C_3**2*R_2**2 + 2*C_3*C_4*R_1*R_2 + 2*C_3*C_4*R_2**2 + C_4**2*R_2**2)),
 'K_BP': C_3*R_1*R_2*R_4*R_6/(C_3*R_1*R_4*R_5 + C_3*R_2*R_4*R_5 + C_4*R_2*R_4*R_5 - C_6*R_2*R_4*R_6 + C_6*R_2*R_5*R_6 + C_6*R_4*R_5*R_6)}

In [20]:
assistant.get_problem_space()

(0, 8, 4)

In [21]:
assistant.kill_variable("R_1", "R_2")
assistant.kill_variable("R_4", "R_2")
assistant.kill_variable("R_5", "R_2")
assistant.kill_variable("C_3", "C_6")
assistant.kill_variable("C_4", "C_6")

True

In [22]:
assistant.metrics_dict

{'Q': sqrt(3)*C_6*R_2**(5/2)*sqrt(R_6)/(3*C_6*R_2**3 + C_6*R_2**2*R_6),
 'wo': sqrt(3)/(3*C_6*sqrt(R_2)*sqrt(R_6)),
 'bandwidth': (3*C_6*R_2**3 + C_6*R_2**2*R_6)/(3*C_6**2*R_2**3*R_6),
 'K_BP': C_6*R_2**3*R_6/(3*C_6*R_2**3 + C_6*R_2**2*R_6)}

In [23]:
assistant.is_metrics_independant()

Jacobian Matrix:
Matrix([[0, -sqrt(3)*sqrt(R_2)*sqrt(R_6)/(3*R_2 + R_6)**2 + sqrt(3)*sqrt(R_2)/(2*sqrt(R_6)*(3*R_2 + R_6)), -3*sqrt(3)*sqrt(R_2)*sqrt(R_6)/(3*R_2 + R_6)**2 + sqrt(3)*sqrt(R_6)/(2*sqrt(R_2)*(3*R_2 + R_6))], [-sqrt(3)/(3*C_6**2*sqrt(R_2)*sqrt(R_6)), -sqrt(3)/(6*C_6*sqrt(R_2)*R_6**(3/2)), -sqrt(3)/(6*C_6*R_2**(3/2)*sqrt(R_6))], [-(R_2 + R_6/3)/(C_6**2*R_2*R_6), 1/(3*C_6*R_2*R_6) - (R_2 + R_6/3)/(C_6*R_2*R_6**2), 1/(C_6*R_2*R_6) - (R_2 + R_6/3)/(C_6*R_2**2*R_6)], [0, -R_2*R_6/(3*R_2 + R_6)**2 + R_2/(3*R_2 + R_6), -3*R_2*R_6/(3*R_2 + R_6)**2 + R_6/(3*R_2 + R_6)]])
Rank: 3
Number of equations: 4
The system is dependent.


False

In [28]:
assistant.get_jacobian_of_metrics()

Jacobian Matrix:
Matrix([[0, -sqrt(3)*sqrt(R_2)*sqrt(R_6)/(3*R_2 + R_6)**2 + sqrt(3)*sqrt(R_2)/(2*sqrt(R_6)*(3*R_2 + R_6)), -3*sqrt(3)*sqrt(R_2)*sqrt(R_6)/(3*R_2 + R_6)**2 + sqrt(3)*sqrt(R_6)/(2*sqrt(R_2)*(3*R_2 + R_6))], [-(R_2 + R_6/3)/(C_6**2*R_2*R_6), 1/(3*C_6*R_2*R_6) - (R_2 + R_6/3)/(C_6*R_2*R_6**2), 1/(C_6*R_2*R_6) - (R_2 + R_6/3)/(C_6*R_2**2*R_6)], [0, -R_2*R_6/(3*R_2 + R_6)**2 + R_2/(3*R_2 + R_6), -3*R_2*R_6/(3*R_2 + R_6)**2 + R_6/(3*R_2 + R_6)]])
Row Reduced Jacobian Matrix:
Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
Dependent Rows (indices): []


(Matrix([
 [                              0, -sqrt(3)*sqrt(R_2)*sqrt(R_6)/(3*R_2 + R_6)**2 + sqrt(3)*sqrt(R_2)/(2*sqrt(R_6)*(3*R_2 + R_6)), -3*sqrt(3)*sqrt(R_2)*sqrt(R_6)/(3*R_2 + R_6)**2 + sqrt(3)*sqrt(R_6)/(2*sqrt(R_2)*(3*R_2 + R_6))],
 [-(R_2 + R_6/3)/(C_6**2*R_2*R_6),                                            1/(3*C_6*R_2*R_6) - (R_2 + R_6/3)/(C_6*R_2*R_6**2),                                                1/(C_6*R_2*R_6) - (R_2 + R_6/3)/(C_6*R_2**2*R_6)],
 [                              0,                                                 -R_2*R_6/(3*R_2 + R_6)**2 + R_2/(3*R_2 + R_6),                                                 -3*R_2*R_6/(3*R_2 + R_6)**2 + R_6/(3*R_2 + R_6)]]),
 Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]),
 [])

In [25]:
assistant.remove_metric("wo")

True

In [26]:
assistant.get_metrics()

dict_keys(['Q', 'bandwidth', 'K_BP'])

In [27]:
assistant.is_metrics_independant()

Jacobian Matrix:
Matrix([[0, -sqrt(3)*sqrt(R_2)*sqrt(R_6)/(3*R_2 + R_6)**2 + sqrt(3)*sqrt(R_2)/(2*sqrt(R_6)*(3*R_2 + R_6)), -3*sqrt(3)*sqrt(R_2)*sqrt(R_6)/(3*R_2 + R_6)**2 + sqrt(3)*sqrt(R_6)/(2*sqrt(R_2)*(3*R_2 + R_6))], [-(R_2 + R_6/3)/(C_6**2*R_2*R_6), 1/(3*C_6*R_2*R_6) - (R_2 + R_6/3)/(C_6*R_2*R_6**2), 1/(C_6*R_2*R_6) - (R_2 + R_6/3)/(C_6*R_2**2*R_6)], [0, -R_2*R_6/(3*R_2 + R_6)**2 + R_2/(3*R_2 + R_6), -3*R_2*R_6/(3*R_2 + R_6)**2 + R_6/(3*R_2 + R_6)]])
Rank: 3
Number of equations: 3
The system is independent.


True

## Inverse Solving

In [ ]:
assistant.relax_design_variable("R_1")
assistant.relax_design_variable("C_6")
assistant.relax_design_variables

In [ ]:
assistant.get_problem_space()

In [ ]:
assistant.reset_design_variable()
assistant.relax_design_variables, assistant.design_variables_dict

In [ ]:
assistant.get_problem_space()

In [ ]:
assistant.is_metrics_independant()

In [27]:
assistant.kill_variable("R_1", "R_2")
assistant.kill_variable("C_3", "C_6")
assistant.kill_variable("C_4", "C_6")



In [ ]:
assistant.metrics_dict

In [ ]:
assistant.get_problem_space()

In [ ]:
# assistant.solve_inverse()

In [ ]:
assistant.design_variables_dict

In [ ]:
sol = sympy.solve(assistant.equations, [
    assistant.design_variables_dict["R_6"],
    assistant.design_variables_dict["C_6"],
    assistant.design_variables_dict["R_4"],
    # assistant.design_variables_dict["R_5"]
    ], dict=True)[0]

sol

In [ ]:
prod = (assistant.metrics_dict["Q"]*assistant.metrics_dict["bandwidth"]).simplify()
prod

In [ ]:
prod/assistant.metrics_dict["wo"]